In [23]:
import pandas as pd 
df=pd.read_csv(r"C:\Users\shrey\Downloads\financial_data.xls")
df.head()
df.drop_duplicates(inplace=True)
df['Sentence'] = df['Sentence'].str.lower()
df.head(3)

,Sentence,Sentiment
0,the geosolutions technology will leverage bene...,positive
1,"$esi on lows, down $1.50 to $2.50 bk a real po...",negative
2,"for the last quarter of 2010 , componenta 's n...",positive


In [43]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout,SpatialDropout1D

In [45]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df["Sentiment"] = le.fit_transform(df["Sentiment"])

In [46]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000, lower=True, oov_token='shreya')
tokenizer.fit_on_texts(df['Sentence'].values)

In [47]:
max_words = len(tokenizer.word_index)

print(max_words)

11544


In [48]:
max_len = max([len(x.split()) for x in df["Sentence"]])
max_len

81

In [49]:
seq =tokenizer.texts_to_sequences(df["Sentence"])

In [50]:
pad_sequences(tokenizer.texts_to_sequences(df["Sentence"]),maxlen=max_len)

array([[   0,    0,    0, ..., 3965,  580,  474],
       [   0,    0,    0, ...,    7,  216, 3967],
       [   0,    0,    0, ...,   68,    3, 3128],
       ...,
       [   0,    0,    0, ...,  388,   94, 1242],
       [   0,    0,    0, ..., 3208,   40,   16],
       [   0,    0,    0, ...,    6,    1,  253]], dtype=int32)

In [51]:
x = pad_sequences(tokenizer.texts_to_sequences(df["Sentence"]),maxlen=max_len)
y = df["Sentiment"]

In [52]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y,num_classes = 82)

In [53]:
print(y[0])

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [54]:
x_train, x_test, y_train,y_test = train_test_split(x,y,test_size=0.25)

In [78]:
early_stopping = tensorflow.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=5,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
)

In [79]:
model = Sequential()
model.add(Embedding(max_words+1, 100))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(6))
model.add(Dense(82, activation='softmax'))

In [80]:
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

In [82]:
fit = model.fit(
    x_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping]
)

Epoch 1/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 8s 58ms/step - accuracy: 0.8777 - loss: 0.2909 - val_accuracy: 0.6552 - val_loss: 1.0575
Epoch 2/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 8s 56ms/step - accuracy: 0.8787 - loss: 0.2714 - val_accuracy: 0.6703 - val_loss: 1.1144
Epoch 3/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 9s 63ms/step - accuracy: 0.8955 - loss: 0.2283 - val_accuracy: 0.6676 - val_loss: 1.1908
Epoch 4/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 9s 67ms/step - accuracy: 0.9070 - loss: 0.2094 - val_accuracy: 0.6408 - val_loss: 1.2715
Epoch 5/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 9s 67ms/step - accuracy: 0.9006 - loss: 0.1921 - val_accuracy: 0.6347 - val_loss: 1.2769
Epoch 6/50
137/137 ━━━━━━━━━━━━━━━━━━━━ 9s 67ms/step - accuracy: 0.9123 - loss: 0.1883 - val_accuracy: 0.6484 - val_loss: 1.4555
Epoch 6: early stopping


In [83]:
y_pred = model.predict(x_test)
from sklearn.metrics import classification_report
import numpy as np

# Convert predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# If y_test is one-hot encoded, convert it back to class labels
y_test_labels = np.argmax(y_test, axis=1)

# Print the classification report
print(classification_report(y_test_labels, y_pred_labels, target_names=['negative', 'neutral', 'positive']))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
              precision    recall  f1-score   support

    negative       0.21      0.17      0.19       222
     neutral       0.69      0.79      0.74       758
    positive       0.75      0.65      0.69       479

    accuracy                           0.65      1459
   macro avg       0.55      0.53      0.54      1459
weighted avg       0.64      0.65      0.64      1459

[[ 37 142  43]
 [ 97 599  62]
 [ 40 129 310]]
